# Бейзлайны и детерминированные алгоритмы item-item

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

np.random.seed(42)

In [144]:
data = pd.read_csv('./data/retail_train.csv')
preds = pd.read_csv('./data/predictions_basic.csv')
# data.head()
# preds.head()

In [160]:
test_size_weeks = 3 # порог недель для сплита временных данных
k = 5 # количество рекомендаций на выходе
ntop_positions = 4000


data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [146]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278490 entries, 0 to 2282324
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 226.0 MB


In [147]:
preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2042 entries, 0 to 2041
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   user_id                 2042 non-null   int64 
 1   actual                  2042 non-null   object
 2   random_recommendation   2042 non-null   object
 3   popular_recommendation  2042 non-null   object
 4   itemitem                2042 non-null   object
 5   cosine                  2042 non-null   object
 6   tfidf                   2042 non-null   object
 7   own_purchases           2042 non-null   object
dtypes: int64(1), object(7)
memory usage: 127.8+ KB


In [148]:
# import ast
# preds['random_recommendation'].apply(lambda s: list(ast.literal_eval(s)))

def str_tolist(row):
    
    """Конвертация строковых ячеек с список
    
    Input
    -----
    row: str list
        Список заковыченных список с разделителями "," и без
    """
    
    list_int = []
    for i in row.strip('][').split():
        list_int.append(int(i.rstrip(',')))
    return list_int


# фарматирование строк

for cat_colname in preds.select_dtypes(include='object').columns:
    preds[cat_colname] = preds.apply(lambda s: str_tolist(s[cat_colname]), axis=1)

### Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [149]:
def weighted_random_recommendation(items_weights, n=5):
    
    """Случайные рекоммендации, вероятность рекомендовать товар прямо пропорциональна логарифму продаж
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()


def get_item_weights(df):
    
    """Ранжирование товаров по весам, пропорциональным логарифму продаж
    
    Input
    -----
    df: pd.DataFrame
    """
    
    weights = df.groupby('item_id')['sales_value'].sum().reset_index()
    weights = weights[weights['sales_value']>1]
    weights['weight']= np.log(weights['sales_value'] + 1e-8)
    weights['weight'] = weights['weight'] / weights['weight'].sum()
    weights.drop('sales_value', axis=1, inplace=True)
    
    return weights

In [150]:
%%time

item_weights = get_item_weights(data_train)

preds['weighted_random_recommendation'] = preds['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=k))
preds.head()

Wall time: 8.53 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[1025411, 13911667, 8184528, 2049325, 894764]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[894752, 836281, 12385586, 2227502, 7441783]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[473690, 15596515, 10341855, 927041, 909479]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]","[910309, 983267, 1116096, 1059864, 974617]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]","[5564325, 884363, 975324, 1019893, 1074521]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [151]:
# Precision - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров юзер купил
# Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [152]:
for col in preds.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(preds.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()))

random_recommendation:  0.0006
popular_recommendation:  0.1552
itemitem:  0.0336
cosine:  0.0353
tfidf:  0.0361
own_purchases:  0.1800
weighted_random_recommendation:  0.0013


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [164]:
# Улучшим бейзлайн присвоения весов товарам

def get_popular_item_weights(df, ntop=5000):
    
    """Ранжирование товаров по весам, пропорциональным логарифму продаж по топовым позициям
    
    Input
    -----
    df: pd.DataFrame
    """
    
    pop_weights = df.groupby('item_id')['sales_value'].sum().reset_index()
    pop_weights.sort_values('sales_value', ascending=False, inplace=True)

    pop_weights = pop_weights[:ntop]
    pop_weights['weight']= np.log(pop_weights['sales_value'] + 1e-3)
    pop_weights['weight'] = pop_weights['weight'] / pop_weights['weight'].sum()
    pop_weights.drop('sales_value', axis=1, inplace=True)
    
    return pop_weights

In [165]:
%%time

# Объединение Weighted Random Recommendation и Popularity-based recommendation

pop_item_weights = get_popular_item_weights(data_train, ntop_positions)

preds['popular_weighted_random_recommendation'] = preds['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=k))
preds.head()

Wall time: 8.72 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,popular_weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[1025411, 13911667, 8184528, 2049325, 894764]","[822629, 10149863, 917384, 975505, 904875]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[894752, 836281, 12385586, 2227502, 7441783]","[9487414, 1615555, 9337674, 6423993, 863447]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[473690, 15596515, 10341855, 927041, 909479]","[981677, 1105137, 983659, 924421, 12456592]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]","[910309, 983267, 1116096, 1059864, 974617]","[9245312, 1007667, 9446848, 5575652, 50509]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]","[5564325, 884363, 975324, 1019893, 1074521]","[1028890, 974108, 930466, 950566, 13774754]"


In [166]:
# Вывод Precision@k
for col in preds.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(preds.apply(lambda row: precision_at_k(row[col], row['actual'], k=k), axis=1).mean()))

random_recommendation:  0.0006
popular_recommendation:  0.1552
itemitem:  0.0336
cosine:  0.0353
tfidf:  0.0361
own_purchases:  0.1800
weighted_random_recommendation:  0.0013
popular_weighted_random_recommendation:  0.0016


In [ ]:
# Item-Item Recommender / ItemKNN